In [ ]:
pip install pycrypto

In [ ]:
import random
import os
import random
from Crypto.Hash import MD5
import base64
from pwn import *
import json


PREFIX = b"CryptoHack Secure Safe"


def xor(a, b):
    return a ^ b

magic1 = 2
magic2 = 11

h1 = 3
h2 = 5
for i in range(2, 2**(random.randint(2, 10))):
    h1 = xor(magic1, xor(h2, xor(xor(h2, xor(h1, h2)), h2)))
    h2 = xor(xor(xor(h1, xor(xor(h2, h1), h1)), h1), magic2)
s = 'CryptoHack Secure Safe'
print(len(s) * 8)
print(h1, h2)

f1 = 'Q3J5cHRvSGFjayBTZWN1cmUgU2FmZTEya5cwTmcqxqoXutB8YALU+RrjW9+jh0vV9XBvWRWkeer2IKlhH1PPLVQKOa3V9kM/TWiJIyh2mZ3YjfXxVEX05hCDpMEncLoFknNISJaRbBm4jPELdb3P0Arfc5wldvhHbM0Et4Axf68='
f2 = 'Q3J5cHRvSGFjbCBTZWN1cmUgU2FmZTEya5cwTmcqxqoXutB8YALU+RrjW9+jh0vV9XBvWRWkeer2IKlhH1PPLVQKOa3V9kM/TWeJIyh2mZ3YjfXxVEX05hCDpMEncLoFknNISJaRbBm4jPELdb3P0Arfc5wldvhHbM0Et4Axf68='


base64_bytes1 = f1.encode('ascii')
message_bytes1 = base64.b64decode(base64_bytes1)

base64_bytes2 = f2.encode('ascii')
message_bytes2 = base64.b64decode(base64_bytes2)

assert(message_bytes2 != message_bytes1)

HOST = 'socket.cryptohack.org'
PORT = 13397
r = remote(HOST, PORT)
data1 = {"option": "insert_key", "key": message_bytes1.hex()}
payload1 = json.dumps(data1)
data2 = {'option': "insert_key", "key": message_bytes2.hex()}
payload2 = json.dumps(data2)
data3 = {"option": "unlock"}
payload3 = json.dumps(data3)
r.recvuntil("safe?\n")

r.sendline(payload1)
print(r.recvline())
r.sendline(payload2)
print(r.recvline())
r.sendline(payload3)
ret = r.recvline()
print(ret)

In [ ]:
pip install pycryptodomex

In [2]:
pip install pkcs1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
import telnetlib
import json
import re
from pkcs1 import emsa_pkcs1_v15
from sage.all import *
import fastecdsa
from fastecdsa.point import Point
from Crypto.Util.Padding import pad

SBOX = [
    0xf0, 0xf3, 0xf1, 0x69, 0x45, 0xff, 0x2b, 0x4f, 0x63, 0xe1, 0xf3, 0x71, 0x44, 0x1b, 0x35, 0xc8,
    0xbe, 0xc0, 0x1a, 0x89, 0xec, 0x3e, 0x1d, 0x3a, 0xe3, 0xbe, 0xd3, 0xcf, 0x20, 0x4e, 0x56, 0x22,
    0xe4, 0x43, 0x9a, 0x6f, 0x43, 0xa9, 0x87, 0x37, 0xec, 0x2, 0x3b, 0x8a, 0x7a, 0x13, 0x7e, 0x79,
    0xcc, 0x92, 0xd7, 0xd1, 0xff, 0x5e, 0xe2, 0xb1, 0xc9, 0xd3, 0xda, 0x40, 0xfb, 0x80, 0xe6, 0x30,
    0x79, 0x1a, 0x28, 0x13, 0x1f, 0x2c, 0x73, 0xb9, 0x71, 0x9e, 0xa6, 0xd5, 0x30, 0x84, 0x9d, 0xa1,
    0x9b, 0x6d, 0xf9, 0x8a, 0x3d, 0xe9, 0x47, 0x15, 0x50, 0xb, 0xe2, 0x3d, 0x3f, 0x1, 0x59, 0x9b,
    0x85, 0xe4, 0xe5, 0x90, 0xe2, 0x2d, 0x80, 0x5e, 0x6b, 0x77, 0xa1, 0x10, 0x99, 0x72, 0x7f, 0x86,
    0x1f, 0x25, 0xa3, 0xea, 0x57, 0x5f, 0xc4, 0xc6, 0x7d, 0x7, 0x15, 0x90, 0xcb, 0x8c, 0xec, 0x11,
    0x9b, 0x59, 0x1, 0x3f, 0x3d, 0xe2, 0xb, 0x50, 0x15, 0x47, 0xe9, 0x3d, 0x8a, 0xf9, 0x6d, 0x9b,
    0xa1, 0x9d, 0x84, 0x30, 0xd5, 0xa6, 0x9e, 0x71, 0xb9, 0x73, 0x2c, 0x1f, 0x13, 0x28, 0x1a, 0x79,
    0x11, 0xec, 0x8c, 0xcb, 0x90, 0x15, 0x7, 0x7d, 0xc6, 0xc4, 0x5f, 0x57, 0xea, 0xa3, 0x25, 0x1f,
    0x86, 0x7f, 0x72, 0x99, 0x10, 0xa1, 0x77, 0x6b, 0x5e, 0x80, 0x2d, 0xe2, 0x90, 0xe5, 0xe4, 0x85,
    0x22, 0x56, 0x4e, 0x20, 0xcf, 0xd3, 0xbe, 0xe3, 0x3a, 0x1d, 0x3e, 0xec, 0x89, 0x1a, 0xc0, 0xbe,
    0xc8, 0x35, 0x1b, 0x44, 0x71, 0xf3, 0xe1, 0x63, 0x4f, 0x2b, 0xff, 0x45, 0x69, 0xf1, 0xf3, 0xf0,
    0x30, 0xe6, 0x80, 0xfb, 0x40, 0xda, 0xd3, 0xc9, 0xb1, 0xe2, 0x5e, 0xff, 0xd1, 0xd7, 0x92, 0xcc,
    0x79, 0x7e, 0x13, 0x7a, 0x8a, 0x3b, 0x2, 0xec, 0x37, 0x87, 0xa9, 0x43, 0x6f, 0x9a, 0x43, 0xe4,
]

def permute(block):
    result = [0 for _ in range(8)]
    for i in range(8):
        x = block[i]
        for j in range(8):
            result[j] |= (x & 1) << i
            x >>= 1
    return result


def substitute(block):
    return [SBOX[x] for x in block]

data1 = [0,0,0,0]
data2 = [0,0,0,0]
state = [16, 32, 48, 80, 80, 96, 112, 128]
for _ in range(2):
    state = permute(state)
    state = substitute(state)
    data1+=state[4:]
    data2+=state[4:]
    state[4:] = [0]*4
    
data2 = data2[:-4]

HOST = "socket.cryptohack.org"
PORT = 13395

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline().decode()
    st = line[line.find('{'):]
    return json.loads(st)

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

tn = telnetlib.Telnet(HOST, PORT)
print(readline())

to_send = json.loads(json.dumps({"a" : bytes(data1).hex(), "b" : bytes(data2).hex()}))
json_send(to_send)

print(json_recv())

In [ ]:
pip install pycryptodome

In [ ]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
import telnetlib
import json
import re
from pkcs1 import emsa_pkcs1_v15
from sage.all import *
import fastecdsa
from fastecdsa.point import Point
from Crypto.Util.Padding import pad

SBOX = [
    0xf0, 0xf3, 0xf1, 0x69, 0x45, 0xff, 0x2b, 0x4f, 0x63, 0xe1, 0xf3, 0x71, 0x44, 0x1b, 0x35, 0xc8,
    0xbe, 0xc0, 0x1a, 0x89, 0xec, 0x3e, 0x1d, 0x3a, 0xe3, 0xbe, 0xd3, 0xcf, 0x20, 0x4e, 0x56, 0x22,
    0xe4, 0x43, 0x9a, 0x6f, 0x43, 0xa9, 0x87, 0x37, 0xec, 0x2, 0x3b, 0x8a, 0x7a, 0x13, 0x7e, 0x79,
    0xcc, 0x92, 0xd7, 0xd1, 0xff, 0x5e, 0xe2, 0xb1, 0xc9, 0xd3, 0xda, 0x40, 0xfb, 0x80, 0xe6, 0x30,
    0x79, 0x1a, 0x28, 0x13, 0x1f, 0x2c, 0x73, 0xb9, 0x71, 0x9e, 0xa6, 0xd5, 0x30, 0x84, 0x9d, 0xa1,
    0x9b, 0x6d, 0xf9, 0x8a, 0x3d, 0xe9, 0x47, 0x15, 0x50, 0xb, 0xe2, 0x3d, 0x3f, 0x1, 0x59, 0x9b,
    0x85, 0xe4, 0xe5, 0x90, 0xe2, 0x2d, 0x80, 0x5e, 0x6b, 0x77, 0xa1, 0x10, 0x99, 0x72, 0x7f, 0x86,
    0x1f, 0x25, 0xa3, 0xea, 0x57, 0x5f, 0xc4, 0xc6, 0x7d, 0x7, 0x15, 0x90, 0xcb, 0x8c, 0xec, 0x11,
    0x9b, 0x59, 0x1, 0x3f, 0x3d, 0xe2, 0xb, 0x50, 0x15, 0x47, 0xe9, 0x3d, 0x8a, 0xf9, 0x6d, 0x9b,
    0xa1, 0x9d, 0x84, 0x30, 0xd5, 0xa6, 0x9e, 0x71, 0xb9, 0x73, 0x2c, 0x1f, 0x13, 0x28, 0x1a, 0x79,
    0x11, 0xec, 0x8c, 0xcb, 0x90, 0x15, 0x7, 0x7d, 0xc6, 0xc4, 0x5f, 0x57, 0xea, 0xa3, 0x25, 0x1f,
    0x86, 0x7f, 0x72, 0x99, 0x10, 0xa1, 0x77, 0x6b, 0x5e, 0x80, 0x2d, 0xe2, 0x90, 0xe5, 0xe4, 0x85,
    0x22, 0x56, 0x4e, 0x20, 0xcf, 0xd3, 0xbe, 0xe3, 0x3a, 0x1d, 0x3e, 0xec, 0x89, 0x1a, 0xc0, 0xbe,
    0xc8, 0x35, 0x1b, 0x44, 0x71, 0xf3, 0xe1, 0x63, 0x4f, 0x2b, 0xff, 0x45, 0x69, 0xf1, 0xf3, 0xf0,
    0x30, 0xe6, 0x80, 0xfb, 0x40, 0xda, 0xd3, 0xc9, 0xb1, 0xe2, 0x5e, 0xff, 0xd1, 0xd7, 0x92, 0xcc,
    0x79, 0x7e, 0x13, 0x7a, 0x8a, 0x3b, 0x2, 0xec, 0x37, 0x87, 0xa9, 0x43, 0x6f, 0x9a, 0x43, 0xe4,
]

def permute(block):
    result = [0 for _ in range(8)]
    for i in range(8):
        x = block[i]
        for j in range(8):
            result[j] |= (x & 1) << i
            x >>= 1
    return result


def substitute(block):
    return [SBOX[x] for x in block]

data1 = [0,0,0,0]
data2 = [0,0,0,0]
state = [16, 32, 48, 80, 80, 96, 112, 128]
for _ in range(2):
    state = permute(state)
    state = substitute(state)
    data1+=state[4:]
    data2+=state[4:]
    state[4:] = [0]*4
    
data2 = data2[:-4]

HOST = "socket.cryptohack.org"
PORT = 13395

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline().decode()
    st = line[line.find('{'):]
    return json.loads(st)

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

tn = telnetlib.Telnet(HOST, PORT)
print(readline())

to_send = json.loads(json.dumps({"a" : bytes(data1).hex(), "b" : bytes(data2).hex()}))
json_send(to_send)

print(json_recv())

In [ ]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
import telnetlib
from Crypto.Cipher import AES
import json
import re
from pkcs1 import emsa_pkcs1_v15
from sage.all import *
import fastecdsa
from fastecdsa.point import Point
from Crypto.Util.Padding import pad

def bxor(a, b):
    return bytes(x ^ y for x, y in zip(a, b))


def hash(data, sig):
    data = pad(data, 16)
    out = sig
#     print(data)
    for i in range(0, len(data), 16):
        blk = data[i:i+16]
#         print(blk)
        out = bxor(AES.new(blk, AES.MODE_ECB).encrypt(out), out)
    return out

HOST = "socket.cryptohack.org"
PORT = 13388

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline().decode()
    st = line[line.find('{'):]
    return json.loads(st)

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

tn = telnetlib.Telnet(HOST, PORT)
print(readline())

to_send = json.loads(json.dumps({"option" : "sign", "message" : bytes([0]*15).hex()}))
json_send(to_send)
sig = json_recv()["signature"]
sig = hash(b'admin=True',bytes.fromhex(sig))

to_send = json.loads(json.dumps({"option" : "get_flag", "message" : (bytes([0]*15)+bytes([1])+b'admin=True').hex(), "signature" : sig.hex()}))
json_send(to_send)
print(json_recv())

In [ ]:
!python3 -m pip install --upgrade pwntools

In [ ]:
conda update python

In [ ]:
!conda install -c conda-forge pycryptodome


In [ ]:
conda update python


In [ ]:
conda install -c conda-forge pycryptodome


In [ ]:
conda install --force-reinstall -c conda-forge pycryptodome


In [ ]:
from pwn import *
from json import dumps, loads
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

(io := remote("socket.cryptohack.org", 13388)).recv()
io.sendline(dumps({"option": "sign", "message": ""}))
io.sendline(
    dumps(
        {
            "option": "get_flag",
            "message": (pad(b"", 16) + b"admin=True").hex(),
            "signature": xor(
                AES.new(pad(b"admin=True", 16), AES.MODE_ECB).encrypt(
                    signature := bytes.fromhex(
                        loads(io.recv().decode().strip())["signature"]
                    )
                ),
                signature,
            ).hex(),
        }
    )
)
print(loads(io.recv().decode())["flag"])

In [ ]:
pip install pycryto

In [2]:
pip install fastecdsa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 777.4 kB/s eta 0:00:001m959.9 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip3 uninstall crypto 
!pip3 uninstall pycrypto 
!pip3 install pycryptodome 

In [1]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
import telnetlib
import json
import re
from pkcs1 import emsa_pkcs1_v15
from sage.all import *
import fastecdsa
from fastecdsa.point import Point
from Crypto.Util.Padding import pad

SBOX = [
    0xf0, 0xf3, 0xf1, 0x69, 0x45, 0xff, 0x2b, 0x4f, 0x63, 0xe1, 0xf3, 0x71, 0x44, 0x1b, 0x35, 0xc8,
    0xbe, 0xc0, 0x1a, 0x89, 0xec, 0x3e, 0x1d, 0x3a, 0xe3, 0xbe, 0xd3, 0xcf, 0x20, 0x4e, 0x56, 0x22,
    0xe4, 0x43, 0x9a, 0x6f, 0x43, 0xa9, 0x87, 0x37, 0xec, 0x2, 0x3b, 0x8a, 0x7a, 0x13, 0x7e, 0x79,
    0xcc, 0x92, 0xd7, 0xd1, 0xff, 0x5e, 0xe2, 0xb1, 0xc9, 0xd3, 0xda, 0x40, 0xfb, 0x80, 0xe6, 0x30,
    0x79, 0x1a, 0x28, 0x13, 0x1f, 0x2c, 0x73, 0xb9, 0x71, 0x9e, 0xa6, 0xd5, 0x30, 0x84, 0x9d, 0xa1,
    0x9b, 0x6d, 0xf9, 0x8a, 0x3d, 0xe9, 0x47, 0x15, 0x50, 0xb, 0xe2, 0x3d, 0x3f, 0x1, 0x59, 0x9b,
    0x85, 0xe4, 0xe5, 0x90, 0xe2, 0x2d, 0x80, 0x5e, 0x6b, 0x77, 0xa1, 0x10, 0x99, 0x72, 0x7f, 0x86,
    0x1f, 0x25, 0xa3, 0xea, 0x57, 0x5f, 0xc4, 0xc6, 0x7d, 0x7, 0x15, 0x90, 0xcb, 0x8c, 0xec, 0x11,
    0x9b, 0x59, 0x1, 0x3f, 0x3d, 0xe2, 0xb, 0x50, 0x15, 0x47, 0xe9, 0x3d, 0x8a, 0xf9, 0x6d, 0x9b,
    0xa1, 0x9d, 0x84, 0x30, 0xd5, 0xa6, 0x9e, 0x71, 0xb9, 0x73, 0x2c, 0x1f, 0x13, 0x28, 0x1a, 0x79,
    0x11, 0xec, 0x8c, 0xcb, 0x90, 0x15, 0x7, 0x7d, 0xc6, 0xc4, 0x5f, 0x57, 0xea, 0xa3, 0x25, 0x1f,
    0x86, 0x7f, 0x72, 0x99, 0x10, 0xa1, 0x77, 0x6b, 0x5e, 0x80, 0x2d, 0xe2, 0x90, 0xe5, 0xe4, 0x85,
    0x22, 0x56, 0x4e, 0x20, 0xcf, 0xd3, 0xbe, 0xe3, 0x3a, 0x1d, 0x3e, 0xec, 0x89, 0x1a, 0xc0, 0xbe,
    0xc8, 0x35, 0x1b, 0x44, 0x71, 0xf3, 0xe1, 0x63, 0x4f, 0x2b, 0xff, 0x45, 0x69, 0xf1, 0xf3, 0xf0,
    0x30, 0xe6, 0x80, 0xfb, 0x40, 0xda, 0xd3, 0xc9, 0xb1, 0xe2, 0x5e, 0xff, 0xd1, 0xd7, 0x92, 0xcc,
    0x79, 0x7e, 0x13, 0x7a, 0x8a, 0x3b, 0x2, 0xec, 0x37, 0x87, 0xa9, 0x43, 0x6f, 0x9a, 0x43, 0xe4,
]

def permute(block):
    result = [0 for _ in range(8)]
    for i in range(8):
        x = block[i]
        for j in range(8):
            result[j] |= (x & 1) << i
            x >>= 1
    return result


def substitute(block):
    return [SBOX[x] for x in block]

data1 = [0,0,0,0]
data2 = [0,0,0,0]
state = [16, 32, 48, 80, 80, 96, 112, 128]
for _ in range(2):
    state = permute(state)
    state = substitute(state)
    data1+=state[4:]
    data2+=state[4:]
    state[4:] = [0]*4
    
data2 = data2[:-4]

HOST = "socket.cryptohack.org"
PORT = 13395

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline().decode()
    st = line[line.find('{'):]
    return json.loads(st)

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

tn = telnetlib.Telnet(HOST, PORT)
print(readline())

to_send = json.loads(json.dumps({"a" : bytes(data1).hex(), "b" : bytes(data2).hex()}))
json_send(to_send)

print(json_recv())

ModuleNotFoundError: No module named 'Crypto.Util.number'